# Bot de extracción de datos ligas de futbol 

## Objetivo

Realizar un Script para recopilar información específica de páginas que contienen estadísticas de alrededor de 5000 partidos de las principales ligas del mundo de manera automatizada.


Esta información será utilizada para realizar un modelo de Machine Learning que per realizar predicciones sobre cuantos tiros de esquina, goles y tarjetas amarillas se producirán después del minuto 75 de un partido.


Se utilizara como principal herramienta la librería Selenium para automatizar la interacción con navegadores web.


In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import pandas as pd
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
import random
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options

cargue de driver de navegador Edge recomendado porque consume menos memoria.

In [3]:
edge_driver_path = 'C:\\Users\\sandr\\OneDrive\\Escritorio\\phyton_2023\\10_futbol\\edge_driver_v1\\msedgedriver.exe'
options = Options()
service = Service(executable_path=edge_driver_path)
driver = webdriver.Edge(service=service, options=options)
url = 'colocar direccion web'
driver.maximize_window()
driver.get(url)
time.sleep(random.randint(3, 6))

Click a botón aceptar de la página para que permita la extracción de datos.

In [5]:
boton_aceptar = driver.find_element(By.XPATH,'colocar xpath')
boton_aceptar.click()
time.sleep(random.randint(2, 5))

función para ejecutar despliegues de listas, se da una espera de hasta 15 segundos para que responda la pagina.

In [6]:
def listas_espera(X_PATH_ES):
    try:
        opciones_ES = WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.XPATH, X_PATH_ES)))
    except WebDriverException as e:
        driver.refresh()
        opciones_ES = WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.XPATH, X_PATH_ES)))
    return opciones_ES

función para ejecutar elementos, se da una espera de hasta 15 segundos para que responda la pagina.

In [7]:
def elemento_(X_PATH_ELEMENT):
    try:
        elemento_ES = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, X_PATH_ELEMENT)))
    except WebDriverException as e:
        driver.refresh()
        elemento_ES = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, X_PATH_ELEMENT)))                
    return elemento_ES

## Extraccion de datos

función para extraer los datos estadísticos de cada partido de futbol.

In [8]:
def varios_():

    time.sleep(random.randint(2, 4))
    X_PATH_ELEMENT = 'colocar xpath'
    boton_local = elemento_(X_PATH_ELEMENT)
    time.sleep(random.randint(2, 4))
    X_PATH_ELEMENT = 'colocar xpath'
    boton_visitante =  elemento_(X_PATH_ELEMENT)
    list_botones = [boton_local,boton_visitante]

    barra = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'colocar xpath')))
    time.sleep(random.randint(2, 3))
    list_barr = [581,-581]

    list_localia = ['local','visitante']
    list_minutes =['70_90','0_90']

    #extraer los nombres de los equipos en orden local y visitante
    X_PATH_ES = 'colocar xpath'
    elementos_3 = listas_espera(X_PATH_ES)
    time.sleep(random.randint(2, 3))
    # Crear una lista para almacenar los valores del atributo title
    valores = []
    # Obtener el valor del atributo title para cada elemento abbr
    for elemento_3 in elementos_3:
        valor = elemento_3.text
        valores.append(valor)

        # Lista original
    lista_original = valores
    # Acceder a la cadena dentro de la lista
    cadena_original = lista_original[0]
    # Separar la cadena en una lista utilizando '\n' como separador
    equipos = cadena_original.split('\n')
    #eliminar all
    equipos.pop(0)
    # Imprimir la lista resultante
    #print(equipos)

    xpath_liga =  'colocar xpath'
    time.sleep(random.randint(2, 3))
    xpath_estadio = 'colocar xpath'
    list_xpath = [xpath_liga,xpath_estadio]
    time.sleep(random.randint(2, 3))
    
    return list_botones,barra,list_barr,list_localia,list_minutes,equipos,list_xpath 



extracción y almacenamiento de tablas por tiempo

In [9]:
def resumen_jugadores():
    elementos = driver.find_elements(By.XPATH,'colocar xpath')
    time.sleep(random.randint(2, 5))   
    valores = []   
    for elemento in elementos:
        valor = elemento.text
        valores.append(valor)
    return valores 

procesamiento de datos para concatenar los nombres y separar los datos numéricos organizando en columnas.

In [10]:
def organizar_data(infor):
    datos = infor
    # Separar las líneas
    lineas = datos.split('\n')
    # Obtener los nombres de columna
    columnas = lineas[0].split()
    # Inicializar lista para datos
    datos_concatenados = []
    # Recorrer las líneas desde la segunda
    for linea in lineas[1:]:
        # Separar elementos por espacio
        elementos = linea.split()
        # Concatenar elementos que tienen texto
        nombre_completo = ' '.join(elementos[:-14])
        # Agregar los elementos restantes
        elementos_restantes = elementos[-14:]
        # Agregar a la lista de datos
        datos_concatenados.append([nombre_completo] + elementos_restantes)
    # Definir nombres de columna
    columnas.insert(0, 'Jugador')
    # Crear DataFrame
    df = pd.DataFrame(datos_concatenados, columns=columnas)
    return df

Procesamiento de datos para restar los datos del minuto 0 al 90 menos el 75 al 90

In [11]:
def resta_0_70_(inf_70_90,inf_0_90):
    df1 = inf_0_90
    df2 = inf_70_90

    # Convertir las columnas a tipos numéricos
    columnas_numericas = df1.columns[df1.columns != 'Jugador']
    df1[columnas_numericas] = df1[columnas_numericas].apply(pd.to_numeric)
    df2[columnas_numericas] = df2[columnas_numericas].apply(pd.to_numeric)

    # Combinar DataFrames según la columna en común
    df_combinado = pd.merge(df1, df2, on='Jugador', suffixes=('_df1', '_df2'))

    # Restar las columnas correspondientes y conservar la columna en común
    columnas_para_restar = df_combinado.columns[df_combinado.columns.str.endswith('_df1')].str.replace('_df1', '')
    df_0_70 = df_combinado[['Jugador']].copy()  # Crear un nuevo DataFrame con la columna en común
    for columna in columnas_para_restar:
        df_0_70 [columna] = df_combinado[f'{columna}_df1'] - df_combinado[f'{columna}_df2']   
    return df_0_70

In [12]:
def extrerxpath(xpath_):
    elementos_3 = driver.find_elements(By.XPATH, xpath_)
    # Crear una lista para almacenar los valores del atributo title
    valores = []
    # Obtener el valor del atributo title para cada elemento abbr
    for elemento_3 in elementos_3:
        valor = elemento_3.text
        valores.append(valor)

    return valores

Procesamiento y extracción de datos estadísticos de cada partido

In [13]:
def extraccion_tabla_datos():
    data_equipos = []
    for boton, localia, equipo in zip(list_botones,list_localia,equipos):
        boton.click()
        time.sleep(random.randint(2, 5))
        time.sleep(1)
        data_temporal = []
        equipo_ = equipo    
        #for para extraer inf de minuto 0 a 90 y 0 a 70
        for barra_m,minutes in zip(list_barras,list_minutes):         
            #barra_.perform()
            (ActionChains(driver).drag_and_drop_by_offset(barra,barra_m,0)).perform()
            time.sleep(random.randint(6, 8))
            result_tabla_jug = resumen_jugadores()

            for cadena in result_tabla_jug:
                # Verificar si la cadena no está vacía
                if cadena:
                    infor = cadena
            informacion = organizar_data(infor)
            data_temporal.append(informacion)
        #funcion para restar a 0-90 0-70     
        inf_70_90 = data_temporal[0]
        inf_0_90 = data_temporal[1]
        informacion_0_70 = resta_0_70_(inf_70_90,inf_0_90)
        data_70_90 = data_temporal[0]
        #agrgar columnas de 0 a 70
        informacion_0_70['minuto'] = '0_70'
        informacion_0_70['localia'] = localia
        informacion_0_70['equipo'] = equipo_
        #agrgar columnas de 70 a 90
        data_70_90['minuto'] ='70_90'
        data_70_90['localia'] = localia
        data_70_90['equipo'] = equipo_
        #unir dataframes por minutos y equipos
        df_final = pd.concat([informacion_0_70,data_70_90], ignore_index=True)
        data_equipos.append(df_final) 
    #unir dataframe con los dos equipos y minutos
    equi_local = data_equipos[0]
    equi_visitante = data_equipos[1]   
    df_equipos_final = pd.concat([equi_local,equi_visitante], ignore_index=True)

    #añadir informacion

    list_inf_adc = []
    for xpath_ in list_xpath:
        xpath_inf = extrerxpath(xpath_)
        list_inf_adc.append(xpath_inf)

    liga_fecha = list_inf_adc[0]
    inf_estadio = list_inf_adc[1]
    df_equipos_final['liga_fecha'] = liga_fecha[0]
    inf_estadio_conc = ', '.join(inf_estadio)
    df_equipos_final['inf_estadio'] = inf_estadio_conc

    return df_equipos_final

    
    

## Movimiento de pagina

In [14]:
def listas_opciones(x_path_opc):
    opciones = driver.find_elements(By.XPATH,x_path_opc)
    time.sleep(random.randint(2, 4))
    return opciones    

In [15]:
def dataframe_final_1():
    nuevos_nombres = ['columnas de estadisticas de partido']
    df_5.columns = nuevos_nombres
    nombre_csv = '.csv'
    separador = '-'
    texto_original = pais_3
    partes = texto_original.split('\n', 1)
    pais_4 = partes[0].strip()
    nombre_archivo = str(valor_sesion) + pais_4 + separador + torneo_5 + nombre_csv
    carpeta = 'colocar xpath'
    sep_archivo = '\\'
    nombre_liga = carpeta+sep_archivo+nombre_archivo
    df_5.to_csv(nombre_liga, index=False)
    print('finalizo la construcción del dataframe')

gatillo_final

In [16]:
X_PATH_ES = 'colocar xpath'
opciones_continentes = listas_espera(X_PATH_ES)
for option in opciones_continentes[4:]:
    if option:
        print("los continentes son: %s" % option.text)
        option.click()
        time.sleep(random.randint(2, 4))
        X_PATH_ES = 'colocar xpath'
        opciones_paises = listas_espera(X_PATH_ES)
        print('*'*50)
        l_fila_pais = 0
        p1=len(opciones_paises)
        #for opcion_pais in opciones_paises[2:]:
        for opcion_pais in range(11,p1):
            #listas_espera(X_PATH_ES)
            X_PATH_ES = 'colocar xpath'
            opciones_paises = listas_espera(X_PATH_ES)
            opciones_paises[opcion_pais].click()
            time.sleep(random.randint(2, 4))
            pais_3 = opciones_paises[opcion_pais].text
            print(f'los paises son: {pais_3}')
            X_PATH_ES = 'colocar xpath'
            opciones_torneos = listas_espera(X_PATH_ES)          
            l_fila_pais = l_fila_pais + 1
            X_PATH_ES = 'colocar xpath'
            urls_torneos = listas_espera(X_PATH_ES)
            for indice_2, url_torneo in enumerate(urls_torneos):
                url_primer_torneo = url_torneo.get_attribute('colocar xpath')
                torneo_5 = url_torneo.text
                print(f'los torneos son: {torneo_5}')
                driver.execute_script(f'window.open("{url_primer_torneo}")')
                for valor_sesion in range(4,5):  
                        
                        time.sleep(random.randint(7, 8))
                        mis_tabs = driver.window_handles
                        driver.switch_to.window(mis_tabs[1])
                        time.sleep(random.randint(7, 8))    
                    
                        list_sesion = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,f'colocar xpath')))
                        temporada = list_sesion.text
                        print(f'lass sesiones son: {temporada}')
                        list_sesion.click()
                        time.sleep(random.randint(3, 6))
                        x_path_opc = 'colocar xpath'
                        c = len(listas_opciones(x_path_opc))               
                        columnas = ['colocar columnas d eestadisticas']
                        df_5 = pd.DataFrame(columns=columnas)
                        
                        for fecha_ in range(0, c):   
                                X_PATH_ELEMENT = 'colocar xpath'
                                boton_desplegar = elemento_(X_PATH_ELEMENT)
                                boton_desplegar.click()
                                time.sleep(random.randint(2, 4))
                                X_PATH_ES = 'colocar xpath'
                                lista_fechas = listas_espera(X_PATH_ES)
                                fecha_2 = lista_fechas[fecha_].text
                                print(f'la fecha es: {fecha_2}')
                                lista_fechas[fecha_].click()
                                time.sleep(random.randint(2, 4))
                                X_PATH_ES = 'colocar xpath'
                                b= len(listas_espera(X_PATH_ES))
                        
                         
                        
                                for partido in range(0, b):
                                        try:
                                            partidos = listas_espera(X_PATH_ES)
                                            partidos[partido].click()
                                            driver.switch_to.window(mis_tabs[1])
                                            time.sleep(random.randint(4, 6))
                                            # aquí entra al final

                                            varios_element = varios_()
                                            list_botones = varios_element[0]
                                            barra = varios_element[1]
                                            list_barras =  varios_element[2]
                                            list_localia = varios_element[3]
                                            list_minutes = varios_element[4]
                                            equipos = varios_element[5]
                                            list_xpath = varios_element[6]
                                            data_final_1 = extraccion_tabla_datos()
                                            data_final_1['Temporadas'] = temporada
                                            data_final_1['Fecha'] = fecha_2
                                            df_5 = pd.concat([df_5,data_final_1], ignore_index=True)


                                            #hasta aqui final
                                            X_PATH_ELEMENT = 'colocar xpath'
                                            datos_1 = elemento_(X_PATH_ELEMENT)
                                            datos_f = datos_1.text
                                            driver.back()
                                            time.sleep(random.randint(4, 6))
                                            print(f'este es el partido {partido} y los datos son:{datos_f}')
                                        except WebDriverException as e:
                                            print('error no hay datos de este partido')
                                            driver.back()
                                            time.sleep(random.randint(4, 6))
                                        
                        dataframe_final_1()            
 
                driver.close()
                time.sleep(random.randint(2, 4))
                driver.switch_to.window(mis_tabs[0])
                if indice_2 == len(urls_torneos) - 1:
                    time.sleep(random.randint(2, 4))
                    x_path_cerrar_1 = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,f'colocar xpath')))
                    x_path_cerrar_1.click()
                    time.sleep(random.randint(2, 4))
                    
                       


               

los continentes son: Europe
**************************************************
los paises son: SPAIN
Primera División
Segunda División
Segunda División RFEF
Copa del Rey
Super Cup
los torneos son: Primera División
lass sesiones son: 2020/2021
la fecha es: Matchday 38
este es el partido 0 y los datos son:All
este es el partido 1 y los datos son:All
este es el partido 2 y los datos son:All
este es el partido 3 y los datos son:All
este es el partido 4 y los datos son:All
este es el partido 5 y los datos son:All
este es el partido 6 y los datos son:All
este es el partido 7 y los datos son:All
este es el partido 8 y los datos son:All
este es el partido 9 y los datos son:All
la fecha es: Matchday 37
este es el partido 0 y los datos son:All
este es el partido 1 y los datos son:All
este es el partido 2 y los datos son:All
este es el partido 3 y los datos son:All
este es el partido 4 y los datos son:All
este es el partido 5 y los datos son:All
este es el partido 6 y los datos son:All
este es el

este es el partido 3 y los datos son:All
este es el partido 4 y los datos son:All
este es el partido 5 y los datos son:All
este es el partido 6 y los datos son:All
este es el partido 7 y los datos son:All
este es el partido 8 y los datos son:All
este es el partido 9 y los datos son:All
la fecha es: Matchday 19
este es el partido 0 y los datos son:All
este es el partido 1 y los datos son:All
este es el partido 2 y los datos son:All
este es el partido 3 y los datos son:All
este es el partido 4 y los datos son:All
este es el partido 5 y los datos son:All
este es el partido 6 y los datos son:All
este es el partido 7 y los datos son:All
este es el partido 8 y los datos son:All
este es el partido 9 y los datos son:All
la fecha es: Matchday 18
este es el partido 0 y los datos son:All
este es el partido 1 y los datos son:All
este es el partido 2 y los datos son:All
este es el partido 3 y los datos son:All
este es el partido 4 y los datos son:All
este es el partido 5 y los datos son:All
este es

este es el partido 2 y los datos son:All
este es el partido 3 y los datos son:All
este es el partido 4 y los datos son:All
este es el partido 5 y los datos son:All
este es el partido 6 y los datos son:All
este es el partido 7 y los datos son:All
este es el partido 8 y los datos son:All
este es el partido 9 y los datos son:All
finalizo la construcción del dataframe
los torneos son: Segunda División


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF677FCBC42+60738]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF677F57EE2+233106]
	(No symbol) [0x00007FF677D6AE0B]
	(No symbol) [0x00007FF677DBA947]
	(No symbol) [0x00007FF677DBAA55]
	(No symbol) [0x00007FF677DFC277]
	(No symbol) [0x00007FF677DDC60F]
	(No symbol) [0x00007FF677DAE201]
	(No symbol) [0x00007FF677DF9528]
	(No symbol) [0x00007FF677DDC233]
	(No symbol) [0x00007FF677DAD568]
	(No symbol) [0x00007FF677DAC394]
	(No symbol) [0x00007FF677DAD101]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF67818878C+1074636]
	(No symbol) [0x00007FF677E30F45]
	(No symbol) [0x00007FF677EC8F6C]
	(No symbol) [0x00007FF677EC0A5D]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF6781871D3+1069075]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF677F61E38+273896]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF677F5CB